##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. To run all the code in the notebook, select **Runtime** > **Run all**. To run the code cells one at a time, hover over each cell and select the **Run cell** icon.

![Run cell icon](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/images/beginner/run_cell_icon.png?raw=1)

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.17.0


If you are following along in your own development environment, rather than [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), see the [install guide](https://www.tensorflow.org/install) for setting up TensorFlow for development.

Note: Make sure you have upgraded to the latest `pip` to install the TensorFlow 2 package if you are using your own development environment. See the [install guide](https://www.tensorflow.org/install) for details.

## Load a dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.05539333, -0.30962867,  0.00950384, -0.36400276, -0.16317308,
        -0.177339  , -0.5651519 ,  0.03727103,  0.28565127, -0.08442949]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class:

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.10598183, 0.0821899 , 0.11308784, 0.07784022, 0.09515317,
        0.09381475, 0.06365699, 0.11627197, 0.14905454, 0.10294877]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.3664331

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss:

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.8597 - loss: 0.4811
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.9549 - loss: 0.1529
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9679 - loss: 0.1085
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9727 - loss: 0.0899
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9787 - loss: 0.0701


The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - 2ms/step - accuracy: 0.9790 - loss: 0.0746


[0.07463552802801132, 0.9789999723434448]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[7.1776284e-08, 9.3719588e-10, 5.5500055e-05, 1.1226929e-03,
        1.6329023e-12, 8.8543345e-08, 1.1192381e-13, 9.9882108e-01,
        1.1396366e-08, 4.3314139e-07],
       [3.5852853e-08, 8.8615780e-05, 9.9986458e-01, 8.3677314e-06,
        1.2142193e-15, 1.0742571e-06, 1.3400536e-07, 6.7589800e-14,
        3.7152218e-05, 2.4263276e-12],
       [4.6527654e-07, 9.9848217e-01, 4.1287713e-04, 2.9134068e-05,
        1.6670372e-05, 2.8643174e-06, 4.8170528e-05, 6.2083604e-04,
        3.8630434e-04, 5.2443721e-07],
       [9.9922550e-01, 1.8424824e-08, 2.6250171e-04, 1.2130323e-06,
        5.5877842e-07, 5.1552520e-06, 4.9789349e-04, 2.7545302e-06,
        1.2989776e-07, 4.1408443e-06],
       [2.5102420e-06, 7.8308275e-09, 8.6165537e-06, 7.3818967e-08,
        9.6395653e-01, 4.8880829e-06, 3.7784477e-07, 8.4218373e-05,
        4.4238064e-07, 3.5942391e-02]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).


In [2]:
import tensorflow as tf
import numpy as np

# Replace ... with actual video data
# For example:
# Assume you have a NumPy array 'video_array' containing your video data.
# Replace ... with video_array
video_data = tf.data.Dataset.from_tensor_slices(video_array)

# Modeli video verilerini işleyecek şekilde değiştir
model = tf.keras.models.Sequential([
  tf

SyntaxError: incomplete input (<ipython-input-2-6baac460d81e>, line 12)

In [3]:
!pip install tensorflow==2.12 # Gerekli TensorFlow sürümü
!pip install open3d # PointNet++ ve VoteNet için
!pip install torch torchvision # PyTorch tabanlı modeller için

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 9.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.7 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.9
    Uninstalling widgetsnbextension-3.6.9:
      Successfully uninstalled widgetsnbextension-3.6.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.22.0 requires numpy>=1.24.0, but you have numpy 1.23

In [5]:
# PointNet++ örneği
import pointnet2
# ... modelinizi yükleyin ve kullanın

# VoteNet örneği
import votenet
# ... modelinizi yükleyin ve kullanın

# Monodepth2 örneği
import monodepth2
# ... modelinizi yükleyin ve kullanın

# DenseDepth örneği
import DenseDepth
# ... modelinizi yükleyin ve kullanın

In [6]:
# 1. DenseDepth modelini klonlayın
!git clone https://github.com/ialhashim/DenseDepth.git

# 2. Modeli yükleyin
%cd DenseDepth
!python download_model.py
import DenseDepth

# 3. Derinlik tahmini yapın
model_type = 'MiDaS_small'
# ... modelinizi başlatın ve bir görüntü üzerinde tahmin yapın

fatal: destination path 'DenseDepth' already exists and is not an empty directory.
/content/DenseDepth
python3: can't open file '/content/DenseDepth/download_model.py': [Errno 2] No such file or directory


In [4]:
!git clone https://github.com/charlesq34/pointnet2.git # PointNet++
!git clone https://github.com/facebookresearch/votenet.git # VoteNet
!git clone https://github.com/nianticlabs/monodepth2.git # Monodepth2
!git clone https://github.com/ialhashim/DenseDepth.git # DenseDepth

Cloning into 'pointnet2'...
remote: Enumerating objects: 315, done.
remote: Total 315 (delta 0), reused 0 (delta 0), pack-reused 315 (from 1)
Receiving objects: 100% (315/315), 647.87 KiB | 16.20 MiB/s, done.
Resolving deltas: 100% (175/175), done.
Cloning into 'votenet'...
remote: Enumerating objects: 134, done.
remote: Total 134 (delta 0), reused 0 (delta 0), pack-reused 134 (from 1)
Receiving objects: 100% (134/134), 845.31 KiB | 15.65 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Cloning into 'monodepth2'...
remote: Enumerating objects: 180, done.
remote: Total 180 (delta 0), reused 0 (delta 0), pack-reused 180 (from 1)
Receiving objects: 100% (180/180), 10.27 MiB | 27.96 MiB/s, done.
Resolving deltas: 100% (89/89), done.
Cloning into 'DenseDepth'...
remote: Enumerating objects: 305, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 305 (delta 2), reused 6 (delta 2), pack-reused 298 (from 1)
Receiving objects: 100% (

In [8]:
!pip install open3d
import open3d as o3d

# Nokta bulutu yükleyin ve görselleştirin
pcd = o3d.io.read_point_cloud("point_cloud.ply")
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] Read PLY failed: unable to open file: point_cloud.ply
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [11]:
!pip install plotly
import plotly.graph_objects as go
import numpy as np # Import numpy for creating example data

# Örnek veri oluşturun (x, y, z koordinatları)
# Gerçek verilerinizi burada kullanmanız gerekir.
x = np.random.rand(100)
y = np.random.rand(100)
z = np.random.rand(100)

# 3B saçılma grafiği oluşturun
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers')])
fig.show()

In [13]:
# Nokta bulutunda gürültü azaltma
pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0) # Remove the 4 spaces indent at the beginning of the line.

In [14]:
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers o

E: Unable to locate package o


In [15]:
!wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
!tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

In [16]:
!git clone -q --depth 1 https://github.com/CMU-Perceptual-Computing-Lab/openpose.git

In [17]:
%cd openpose
!sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' CMakeLists.txt
!mkdir build
%cd build
!cmake ..
!make -j`nproc`

/content/DenseDepth/openpose
/content/DenseDepth/openpose/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- GCC detected, adding compile flags
-- GCC detected, adding compile flags
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - found
-- Found Threads: TRUE  
-- Found CUDA: /usr/local/cuda (found version "12.2") 
-- Building with CUDA.
-- CUDA detected: 12.2
-- Found cuD

In [19]:
!cd .. && python3 setup.py install --user




python3: can't open file '/content/DenseDepth/openpose/setup.py': [Errno 2] No such file or directory


In [26]:
!apt-get install blender


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  blender-data fonts-dejavu fonts-dejavu-core fonts-dejavu-extra libdcmtk16 libembree3-3
  libfftw3-double3 libjemalloc2 liblog4cplus-2.0.5 libopencolorio1v5 libopenimageio2.2
  libopenvdb8.1 libosdcpu3.4.4 libpugixml1v5 libraw20 libspnav0 libsquish0 libtbb2
  libtinyxml2.6.2v5 libwrap0 libyaml-cpp0.7
Suggested packages:
  libfftw3-bin libfftw3-dev spacenavd
The following NEW packages will be installed:
  blender blender-data fonts-dejavu fonts-dejavu-core fonts-dejavu-extra libdcmtk16 libembree3-3
  libfftw3-double3 libjemalloc2 liblog4cplus-2.0.5 libopencolorio1v5 libopenimageio2.2
  libopenvdb8.1 libosdcpu3.4.4 libpugixml1v5 libraw20 libspnav0 libsquish0 libtbb2
  libtinyxml2.6.2v5 libwrap0 libyaml-cpp0.7
0 upgraded, 22 newly installed, 0 to remove and 49 not upgraded.
Need to get 116 MB of archives.
After this operation, 706 MB of add

In [27]:
!pip install moviepy

In [28]:
!pip install opencv-python

In [36]:
import tensorflow as tf
import numpy as np

def style_transfer(content_image, style_image):
  content_image = tf.image.convert_image_dtype(content_image, dtype=tf.float32)
  style_image = tf.image.convert_image_dtype(style_image, dtype=tf.float32)

  # Görüntüleri modele uygun boyuta getirin
  content_image = tf.image.resize(content_image, (256, 256))
  style_image = tf.image.resize(style_image, (256, 256))

  # Modeli çalıştırın ve stilize edilmiş görüntüyü alın
  stylized_image = hub_model(tf.constant(content_image[np.newaxis, ...]),
                             tf.constant(style_image[np.newaxis, ...]))[0]

  # Görüntüyü orijinal veri türüne geri dönüştürün
  stylized_image = tf.image.convert_image_dtype(stylized_image, dtype=tf.uint8)

  return stylized_image

In [37]:
from PIL import Image

content_image = Image.open("content_image.jpg")
style_image = Image.open("style_image.jpg")

stylized_image = style_transfer(content_image, style_image)

# Stilize edilmiş görüntüyü kaydedin veya görüntüleyin
stylized_image.save("https://cdn.prod.website-files.com/645cec60ffb18d5ebb37da4b/64a69e4b0be51cace7819e1b_3vr_9kFB3DmHzG4IBX7BEa8qSLEC0YgUTp2b0RppAYEvDbQfhDkkQtaDSuStx7BvgstN6EYvuYCrpW50FVQwC5pImTaTgFA4NpcNUauhgiC0P8CDUcYBjTJLfNPMio0sHTIn1Sz_nhmtqsuUYS9uv-o.png")
stylized_image.show()

FileNotFoundError: [Errno 2] No such file or directory: '/content/DenseDepth/openpose/build/openpose/build/openpose/build/openpose/build/openpose/build/content_image.jpg'

In [38]:
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev

Selecting previously unselected package liblmdb0:amd64.
(Reading database ... 125985 files and directories currently installed.)
Preparing to unpack .../00-liblmdb0_0.9.24-1build2_amd64.deb ...
Unpacking liblmdb0:amd64 (0.9.24-1build2) ...
Selecting previously unselected package libgflags2.2.
Preparing to unpack .../01-libgflags2.2_2.2.2-2_amd64.deb ...
Unpacking libgflags2.2 (2.2.2-2) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../02-libgflags-dev_2.2.2-2_amd64.deb ...
Unpacking libgflags-dev (2.2.2-2) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../03-libgoogle-glog0v5_0.5.0+really0.4.0-2_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.5.0+really0.4.0-2) ...
Selecting previously unselected package libunwind-dev:amd64.
Preparing to unpack .../04-libunwind-dev_1.3.2-2build2.1_amd64.deb ...
Unpacking libunwind-dev:amd64 (1.3.2-2build2.1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to un

In [39]:
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
!wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
!tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
!git clone -q --depth 1 https://github.com/CMU-Perceptual-Computing-Lab/openpose.git
%cd openpose
!sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' CMakeLists.txt
!mkdir build
%cd build
!cmake ..
!make -j`nproc`
!cd .. && python3 setup.py install --user

/content/DenseDepth/openpose/build/openpose/build/openpose/build/openpose/build/openpose/build/openpose
/content/DenseDepth/openpose/build/openpose/build/openpose/build/openpose/build/openpose/build/openpose/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- GCC detected, adding compile flags
-- GCC detected, adding compile flags
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_cre